### P3. Interpretabilidad.
****************************


   1. Parte 1

In [1]:
# librerías usadas
import torch
from torchvision import models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# iniciar datos
def load(path):
    image_rgb =Image.open(path).convert("RGB")
    return image_rgb



# transformaciones compose
transformers=transforms.Compose(
    [transforms.Resize(299),
     transforms.CenterCrop(229),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])         
    ]
)

  2. Parte 2.

In [ ]:
#imagen sde prueba
%%capture
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

### cargo el modelo inception_v3
inception_v3_net = torch.hub.load('pytorch/vision:v0.6.0', 'inception_v3', pretrained=True)

###genero el modelo 
inception_v3_net.eval()


In [ ]:
#transformo los datos
input_image = Image.open(filename)
plt.imshow(np.asarray(input_image))

input_tensor = transformers(input_image)
input_batch = input_tensor.unsqueeze(0)

# # move the input and model to GPU for speed if available
# if torch.cuda.is_available():
#     input_batch = input_batch.to('cuda')
#     model.to('cuda')

# with torch.no_grad():
#   output = model(input_batch)

#predicción del modelo

if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    inception_v3_net.to('cuda')
    
with torch.no_grad():
  out = inception_v3_net(input_batch).cpu()

In [ ]:
from keras.applications.imagenet_utils import decode_predictions
labs = decode_predictions(out.numpy(),top=1)

print('Nombre de la clase predicha: ', labs[0][0][0],'\n')
print('Descripción de la clase: ', labs[0][0][1],'\n')
print('Valor del puntaje de la red: ',labs[0][0][2])

  3. Segmente la imagen de control utilizando la función ```slic``` del módulo ```skimage.segmentation```, para los parámetros ```start_label=0```, ```n_segments=80```. El resultado de la segmentación es un arreglo de dimensión 299&times;299 que asigna una categoría para cada píxel de la imágen procesada. Todos los pixeles en la imágen que comparten etiqueta conforman un súper-píxel dentro de la imágen. Utilice la función ```mark_boundaries``` del módulo ```skimage.segmentation``` en conjunto con ```imshow``` del módulo ```skimage.io``` para visualizar los bordes inducidos por el conjunto de super-pixeles. 

In [ ]:
import skimage.segmentation as sks
from skimage.io import imshow as ims

imagen_np = np.asarray(input_image)
imagen_seg = sks.slic(imagen_np, n_segments=80)

print('Imágen segmentada:\n')
ims(imagen_seg)

In [ ]:
print('Imagen original:\n')
plt.imshow(imagen_np)

In [ ]:
imagen_bnd = sks.mark_boundaries(imagen_np,imagen_seg, color = (0,0,0), outline_color=(0,1,0))
print('Imágen con los super-pixeles marcados en sus bordes:\n')
ims(imagen_bnd)

Al representar una imágen _x_ por medio de la presencia y ausencia de súper-pixeles se logra una representación interpretable _x'_ según un vector de entradas binarias.

Genere perturbaciones en la imágen de control, para esto siga los siguientes pasos:

  4. Defina un número de _perturbaciones_ a realizar (al menos 1000). Cada perturbación consiste en un arreglo binario, donde cada componente es asociada a un súper-pixel. Estos arreglos serán las representaciones interpretables de la imagen de control (_x'_ asociado a _x_). Considere cada entrada del arreglo de perturbaciones como una variable **Bernoulli** con _p=0.5_.

In [10]:
#probaremos con 1000 perturbaciones
n_per = 1000
ind = np.array([(i in imagen_seg) for i in range(80)])
n_clases = len(ind[ind == True])

M = np.zeros([n_clases,n_per])

#binomial(1,p)=bernoulli(p), asigno en una matriz de n_clases x n_perturbaciones donde cada fila corresponde al valor de la clase en esa perturbación.
np.random.seed(10)
for i in range(n_per):
  M[:,i] = np.random.binomial(1,0.5,n_clases)

  5. Genere tantas versiones perturbadas de la imagen de control como perturbaciones haya conseguido. Obtener una imagen perturbada consiste en asignar el valor 0 en cada canal de color en aquellos píxeles cuyos super-pixeles asociados tengan su componente nula en el vector de perturbaciones. Obtenga una visualización de una imágen perturbada.

In [11]:
#generando imágenes perturbadas.

n_x, n_y, n_colores= np.shape(imagen_np) #coordenadas de la imagen normal.

imag_perturb = [] #guardar imágenes perturbadas. guardar en formato Image


for p in range(n_per):
  #para cada perturbación hago una copia de la imagen
  image_copy = imagen_np.copy()
  for i in range(n_x):
    for j in range(n_y):
      pos = imagen_seg[i,j]
      if M[pos,p] == 0:
        image_copy[i,j,:] = [0,0,0]
  
  imag_perturb.append(image_copy)


In [ ]:
# Mostrar una imágen cualquiera
plt.imshow(imag_perturb[425])

  6. Haga predicciones sobre las imágenes perturbadas utilizando la red ```inception_v3```. Asocie el valor 1 como etiqueta a las imágenes perturbadas  que sean clasificadas a la misma categoría de la imágen de control y 0 en caso contrario, el arreglo binario correspondiente se denotará _y_.

In [15]:
#función que evalúa una imágen y la compara con una etiqueta según la predicción
def Evaluación_red(imag_array, nombre_cat, print_ = False):
  input_image = Image.fromarray(imag_array)
  input_tensor = transformers(input_image)
  input_batch = input_tensor.unsqueeze(0)

  if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    inception_v3_net.to('cuda')
    
  with torch.no_grad():
    out = inception_v3_net(input_batch).cpu()
  
  labs = decode_predictions(out.numpy(),top=1)
  
  if print_:
    print(labs[0][0][0])
  
  if labs[0][0][0] == nombre_cat:
    return 1
  else:
    return 0

In [ ]:
# nombre de la categoría correcta.
label = labs[0][0][0]
label

In [ ]:
#Prueba de la función
Evaluación_red(imag_perturb[20], label, print_ = True) #no acertó

In [18]:
#Generación del vector Y
y =[]
for i in range(n_per):
  y.append(Evaluación_red(imag_perturb[i],label))

  7. Calcule &pi;<sub>_x_</sub> según la expresión (3). Para ello, obtenga la distancia del coseno entre las perturbaciones asociadas a cada imágen perturbada y el vector de perturbación de la imágen de control _x_ según lo indica (4).

Tomamos
$$
x' = (1,\cdots,1)
$$
como el vector que identifica a la imagen original $x$. Entonces, si $N$ es la cantidad de clases encontradas en la segmentación, entonces:

$$
x' \in \{0,1\}^N,\hspace{0.4cm}\text{y }\hspace{0.4cm}||x'|| = \sqrt{N}
$$

In [129]:
# Definimos la distancia.
def dist_x(z):
  n = len(z)
  a = np.sum(z)
  norm_x = np.sqrt(n)
  norm_z = np.sqrt(np.sum(z**2))

  d = 1 - (a/(norm_x*norm_z))
  if norm_z == 0:
    return 1
  else:
    return d

# Definimos la distancia coseno:

def pi_x(z,sigma):
  d = dist_x(z)

  return np.exp((-d**2) / (sigma**2))

In [ ]:
# Primera perturbación:
per_0 = M[:,0]
np.shape(per_0)

In [ ]:
# Prueba de las funciones anteriores para sigma=1.
pi_x(per_0, sigma=1)

Una vez obtenido el conjunto de reprsentaciones para la imágen de control $x$ y el vector de pesos asociados &pi;<sub>$x$</sub>, se pasa a minimizar la función de fidelidad, para esto:

  8. Genere un conjunto de entrenamiento $D_p$. Este consta de vectores de perturbación como observaciones. La variable de respuesta será el arreglo $y$ generado anteriormente.

In [22]:
D_p = [M[:,i] for i in range(n_per)]

# y definido anteriormente.

  9. Utilice la clase ```LogisticRegression``` del módulo ```sklearn.linear_model``` para entrenar un clasificador sobre el conjunto de entrenamiento $D_p$. Haga uso de $\pi_x$ al momento de usar el método ```.fit()```. ¿Es posible agregar una medida de complejidad $\Omega(g)$ con este esquema?

In [23]:
from sklearn.linear_model import LogisticRegression
sigma = 0.25
weight = np.array([pi_x(D_p[i],sigma) for i in range(n_per)])

reg = LogisticRegression().fit(D_p,y, sample_weight=weight)

In [ ]:
reg.coef_

   10. Utilice los coeficientes del clasificador anterior para inferir los súper-índices de mayor importancia en la clasificación de la imágen de control.

Dado que la regresión logística vincula a la variable $X$ con la respuesta $Y$ de una forma no lineal, no se analizarán a los coeficientes por el tamaño de su magnitud (módulo) como en la regresión lineal. En cambio, la importancia de la variable estará dada por un factor exponencial del coeficiente. Es decir, si $\beta_i$ es el coeficiente asociado al súper-píxel $i$, entonces la importancia de tal súperpixel se puede medir mediante:

$$
e^{\beta_i}
$$

In [ ]:
coef = np.array(reg.coef_[0])

#región de corte al valor de ln(1.4)
plt.figure(figsize = (10,7))
plt.plot(coef, np.exp(coef), '*', label = 'Importancia del coeficiente en el modelo')
plt.axhline(y=1.4, xmin=0, xmax=1, color = 'r')
plt.legend()

In [ ]:
sup_pix = coef[coef > np.log(1.4)]
Importantes = [i for i in range(54) if coef[i] in sup_pix]
print(Importantes)

In [53]:
# Generamos una imágen que contenga a los superpixéles más importantes:
M_reg = np.zeros(54)
for i in range(54):
  if i in Importantes:
    M_reg[i] = 1


image_new = imagen_np.copy()
for i in range(n_x):
  for j in range(n_y):
    pos = imagen_seg[i,j]
    if M_reg[pos] == 0:
      image_new[i,j,:] = [0,0,0]

In [ ]:
plt.imshow(image_new)

A medida que aumentamos el criterio de elección de los super-pixeles (mayores coeficientes) notamos que los pixeles que se mantienen siempre entre aquellos que el modelo necesita ver para tomar una buena desición son aquellos que están cercanos a la cara del perro (en este caso). Esto es explicado por el nivel de detalle (y por lo tanto, de información) que contiene la imágen y que la diferencia de otras que puedan poseer colores u otras características similares.

La segmentación antes utilizada se hace de _manera espacial_. Es decir, se realiza una clusterización sobre la escala de grises según su posición en la imágen. Del procediminto recién explicado para implementar **LIME** reemplace la etapa de segmentación de la imágen por 2 segmentaciones espaciales utilizando 2 modelos de clustering a su elección, para ello:

  11. Clusterice sobre un conjunto de entrenamiento $X$ con $299^2$ observaciones, es decir, una observación por píxel en la imagen del control escalada. Cada Observación de $X$ consta de 3 componentes, donde la primera y segunda son espaciales (posición del píxel en la imágen) y la última es el valor de intensidad asociado al píxel (escala de grises). Utilice los cluster descubiertos para generar súper-píxeles.

In [ ]:
# Algoritmo clusterizador a ocupar: K-means
# Luego de cargar la imágen en 2d (escala de grises), se genera una matriz de datos donde la fila corresponde al vecrtor [posición_x, posición_y, color_de_(x,y)]

from sklearn.cluster import KMeans
newsize = (299,299)
image_ = np.asarray(Image.open(filename).resize(newsize).convert('RGB'))
image_2d = np.mean(image_, axis=2, dtype=np.uint)
np.shape(image_2d)

In [91]:
X_km = []

for i in range(299):
  for j in range(299):
    X_km.append([i,j, image_2d[i,j]])

In [ ]:
kmean = KMeans().fit_predict(X_km)
np.shape(kmean) # vector de etiquetas para cada vector de X_km, necesario reordenar en matrix de 299x299 para graficar vecindades

In [ ]:
kmean_pix = kmean.reshape([299,299])
kmean_pix.max() +1 #cantidad de clases

In [ ]:
imagen_bnd2 = sks.mark_boundaries(image_,kmean_pix, color = (0,0,0), outline_color=(0,1,0))
ims(imagen_bnd2)

Podemos observar que la mayor cantidad de clusters se concentran en la cara del perro, lugar que, según el modelo anterior, era imprescindible para que la red pueda reconocer la imágen. Clusterización reconoce el pasto, el fondo y el cuerpo del perro como secciones distintas (esto es un poco evidente, puesto que los colores que predominaban en la imágen eran notoriamente distintos, por lo tanto el valor en la imágen también).
Veamos la importancia de cada cluster visto como segmentaciones de los pasos anteriores.

  12. Aplique el esquema **LIME** desarrollado anteriormente sobre sus súper-pixeles.

In [100]:
#probaremos con 1000 perturbaciones denuevo

n_clases_km = kmean_pix.max() +1

M2 = np.zeros([n_clases_km,n_per])

#binomial(1,p)=bernoulli(p), asigno en una matriz de n_clases x n_perturbaciones donde cada fila corresponde al valor de la clase en esa perturbación.
np.random.seed(10)
for i in range(n_per):
  M2[:,i] = np.random.binomial(1,0.5,n_clases_km)

In [112]:
#generando imágenes perturbadas.

imag_perturb_km = [] #guardar imágenes perturbadas. guardar en formato Image


for p in range(n_per):
  #para cada perturbación hago una copia de la imagen
  image_copy = image_.copy()
  for i in range(299):
    for j in range(299):
      pos = kmean_pix[i,j]
      if M2[pos,p] == 0:
        image_copy[i,j,:] = [0,0,0]
  
  imag_perturb_km.append(image_copy)


In [ ]:
# Mostrar una imágen cualquiera 
plt.imshow(imag_perturb_km[640])

In [117]:
#Generación del vector Y
y_km =[]
for i in range(n_per):
  y_km.append(Evaluación_red(imag_perturb_km[i],label))

In [ ]:
D_km = [M2[:,i] for i in range(n_per)]
weight_km = np.array([pi_x(D_km[i],sigma) for i in range(n_per)])

reg_km = LogisticRegression().fit(D_km,y_km, sample_weight=weight_km)

In [ ]:
coef_km = np.array(reg_km.coef_[0])

#región de corte al valor de ln(1.4)
plt.figure(figsize = (10,7))
plt.plot(coef_km, np.exp(coef_km), '*', label = 'Importancia del coeficiente en el modelo')
# plt.axhline(y=1.4, xmin=0, xmax=1, color = 'r')
plt.legend()

Podemos ver sólo un coeficiente positivo (mayor a 6), por lo tanto al aplicar la exponencial, separa brúscamente este coeficiente de los demáses que los sitúa por debajo de 1.

In [ ]:
sup_pix_km = coef_km[coef_km > np.log(100)]
Importantes_km = [i for i in range(8) if coef_km[i]> np.log(100)]
print(Importantes_km,'\n')

# Generamos una imágen que contenga a los superpixéles más importantes:

coef_image = image_.copy()
for i in range(299):
  for j in range(299):
    if kmean_pix[i,j] not in Importantes_km:
      coef_image[i,j,:] = [0,0,0]

plt.imshow(coef_image)

Al igual que la parte anterior, el aspecto más importante de la imágen es la forma que presenta la cara del perro, sin embargo (y no es muy notorio) no considera detalles de la cada, como los ojos, la nariz y el ocico. Esto puede ser por el color negro que presentan, lo que hace que anular el cluster en esa zona no presente mayor diferencia en el modelo, pues el color no cambia drásticamente como el resto del cuerpo. Bajo esa lógica es claro que el modelo de regesión no dará peso de importancia a aquellos cluster.